In [1]:
from collections import defaultdict, deque
from functools import lru_cache

In [2]:
with open("../data/2025/day7.txt") as f:
    lines = f.read().splitlines()

In [3]:
columns = defaultdict(defaultdict)
initial_beam = None
max_depth = len(lines)-1

In [4]:
for y, row in enumerate(lines):
    for x, tile in enumerate(list(row)):
        if tile == '^': columns[x][y] = tile
        elif tile == 'S': initial_beam = x + y*1j

In [5]:
def first_down_obstacle_for_coord(xy: complex):
    for col_depth in columns[int(xy.real)]:
        if col_depth > xy.imag: return col_depth
    return max_depth

In [6]:
def simulate_part1():
    beam_candidates = deque([initial_beam])
    beams = defaultdict(list)
    splitter_activations = set()

    while beam_candidates:
        new_beam = beam_candidates.popleft()

        # If we collide with any existing beam segments, skip
        if any([int(new_beam.imag) in r for r in beams[int(new_beam.real)]]): continue

        # Find the end of this beam (a splitter or exiting the manifold)
        next_splitter = new_beam.real + (depth := first_down_obstacle_for_coord(new_beam)) * 1j
        beam_segment = range(int(new_beam.imag), int(next_splitter.imag) + (1 if depth == max_depth else 0))
        beams[int(new_beam.real)].append(beam_segment)

        for side in (-1, 1) if depth != max_depth else (): # right/left of splitter (not exiting manifold)
            beam_candidates.append(next_splitter + side)
            splitter_activations.add(next_splitter)

    return len(splitter_activations)

print(f"Part 1: {simulate_part1()}")

Part 1: 1516


In [7]:
@lru_cache(maxsize=None)
def combinations_from_splitter(splitter: complex) -> int:
    beam_left = splitter - 1
    left_splitter = (beam_left.real + (left_depth := first_down_obstacle_for_coord(beam_left)) * 1j)
    left_splitter = 1 if left_depth == max_depth else combinations_from_splitter(left_splitter)

    beam_right = splitter + 1
    right_splitter = (beam_right.real + (right_depth := first_down_obstacle_for_coord(beam_right)) * 1j)
    right_splitter = 1 if right_depth == max_depth else combinations_from_splitter(right_splitter)

    return left_splitter + right_splitter

print(f"Part 2: {combinations_from_splitter(initial_beam)}")

Part 2: 1393669447690
